# NB3: Deep Learning Models (LSTM Sequence Models)

**Note**: This notebook uses the filename `NB3` for historical continuity, but internally refers to **Stage 7 (ML7)** following the refactoring to distinguish modeling stages from Jupyter notebook numbering.

**Purpose**: Demonstrate NB3 deep learning results using LSTM sequence models with deterministic training and calendar-based cross-validation.

**Pipeline**: practicum2-nof1-adhd-bd v4.1.x  
**Participant**: P000001  
**Snapshot**: 2025-11-07

This notebook:
1. Loads NB3 outputs (training logs, metrics, predictions)
2. Visualizes training curves (loss, accuracy)
3. Shows per-fold performance metrics
4. Compares NB3 vs NB2 baseline
5. Analyzes sequence predictions and attention patterns
6. Provides markdown commentary on results

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import warnings

warnings.filterwarnings('ignore')

# Configuration
PARTICIPANT = "P000001"
SNAPSHOT = "2025-11-07"
REPO_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

AI_BASE = REPO_ROOT / "data" / "ai" / PARTICIPANT / SNAPSHOT
NB2_DIR = AI_BASE / "nb2"
ML7_DIR = AI_BASE / "ml7"

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 6)

print(f"Repository root: {REPO_ROOT}")
print(f"ML7 (Stage 7) outputs: {ML7_DIR}")

## 1. Load ML7 Results

In [ ]:
if not ML7_DIR.exists():
    print("=" * 80)
    print("❌ ML7 (Stage 7) OUTPUTS NOT FOUND")
    print("=" * 80)
    print(f"\nRequired directory missing: {ML7_DIR}")
    print("\n📋 To generate ML7 (Stage 7) deep learning model results, run:")
    print(f"\n   make nb3 PID={PARTICIPANT} SNAPSHOT={SNAPSHOT}")
    print("\n📝 Note: This requires completed ETL stages 0-4 and NB2 (stage 5-6)")
    print("   If you haven't run the pipeline yet, use:")
    print(f"   make pipeline PID={PARTICIPANT} SNAPSHOT={SNAPSHOT}")
    print("\n💡 Check NB0_DataRead.ipynb to see which stages are complete")
    print("=" * 80)
    raise FileNotFoundError(f"ML7 (Stage 7) outputs not ready. See instructions above.")

# List available files
ml7_files = list(ML7_DIR.glob("*"))
print(f"\nFound {len(ml7_files)} files in ML7 (Stage 7) directory:")
for f in sorted(ml7_files)[:10]:
    print(f"  {f.name}")

# Load training logs (JSON)
log_files = list(ML7_DIR.glob("*training_log*")) + list(ML7_DIR.glob("*history*"))
if log_files:
    log_file = log_files[0]
    print(f"\nLoading training log: {log_file.name}")
    with open(log_file, 'r') as f:
        training_log = json.load(f)
    print(f"✓ Training log loaded: {len(training_log.get('epochs', []))} epochs")
else:
    training_log = None
    print("\n⚠️  No training log found")

# Load metrics summary
metrics_file = ML7_DIR / "metrics_summary.csv"
if metrics_file.exists():
    df_metrics = pd.read_csv(metrics_file)
    print(f"\n✓ Loaded metrics_summary.csv: {df_metrics.shape}")
else:
    # Try per-fold metrics

## 2. Training Curves Visualization

In [ ]:
if training_log is not None:
    # Extract training history
    try:
        if 'history' in training_log:
            hist = training_log['history']
        elif 'epochs' in training_log:
            hist = training_log['epochs']
        else:
            hist = training_log
        
        # Expected keys: loss, val_loss, accuracy, val_accuracy
        if 'loss' in hist and 'val_loss' in hist:
            fig, axes = plt.subplots(1, 2, figsize=(16, 6))
            
            epochs = range(1, len(hist['loss']) + 1)
            
            # Loss curves
            axes[0].plot(epochs, hist['loss'], 'b-', linewidth=2, label='Training Loss', alpha=0.8)
            axes[0].plot(epochs, hist['val_loss'], 'r--', linewidth=2, label='Validation Loss', alpha=0.8)
            axes[0].set_xlabel('Epoch', fontweight='bold')
            axes[0].set_ylabel('Loss', fontweight='bold')
            axes[0].set_title('Training and Validation Loss', fontweight='bold', fontsize=14)
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            
            # Accuracy curves (if available)
            if 'accuracy' in hist and 'val_accuracy' in hist:
                axes[1].plot(epochs, hist['accuracy'], 'b-', linewidth=2, label='Training Accuracy', alpha=0.8)
                axes[1].plot(epochs, hist['val_accuracy'], 'r--', linewidth=2, label='Validation Accuracy', alpha=0.8)
                axes[1].set_xlabel('Epoch', fontweight='bold')
                axes[1].set_ylabel('Accuracy', fontweight='bold')
                axes[1].set_title('Training and Validation Accuracy', fontweight='bold', fontsize=14)
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            print("\n✓ Training curves visualization complete")
            print(f"   Final training loss: {hist['loss'][-1]:.4f}")
            print(f"   Final validation loss: {hist['val_loss'][-1]:.4f}")
        else:
            print(f"\n⚠️  Expected keys not found. Available: {list(hist.keys())}")
    except Exception as e:
        print(f"\n⚠️  Error plotting training curves: {e}")
else:
    print("\n⚠️  Training log not available for visualization")

## 3. ML7 Performance Metrics (Per-Fold)

In [ ]:
if df_metrics is not None and not df_metrics.empty:
    metric_cols = ['macro_f1', 'weighted_f1', 'balanced_accuracy', 'auroc_ovr', 'cohens_kappa']
    available_metrics = [m for m in metric_cols if m in df_metrics.columns]
    
    if available_metrics:
        fig, axes = plt.subplots(1, len(available_metrics), figsize=(16, 5))
        if len(available_metrics) == 1:
            axes = [axes]
        
        for idx, metric in enumerate(available_metrics):
            if 'fold' in df_metrics.columns:
                fold_values = df_metrics.groupby('fold')[metric].mean()
                axes[idx].bar(fold_values.index, fold_values.values, alpha=0.7, color='steelblue')
                axes[idx].axhline(fold_values.mean(), color='red', linestyle='--', 
                                  linewidth=2, label=f'Mean: {fold_values.mean():.3f}')
                axes[idx].set_xlabel('Fold', fontweight='bold')
            else:
                axes[idx].bar([metric], [df_metrics[metric].mean()], alpha=0.7, color='steelblue')
            
            axes[idx].set_ylabel(metric.replace('_', ' ').title(), fontweight='bold')
            axes[idx].set_title(f'{metric.upper()} (NB3)', fontweight='bold')
            axes[idx].set_ylim([0, 1])
            axes[idx].legend()
            axes[idx].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print("\n✓ ML7 (Stage 7) performance metrics visualization complete")
        print("\nSummary Statistics:")
        for m in available_metrics:
            print(f"  {m}: {df_metrics[m].mean():.3f} ± {df_metrics[m].std():.3f}")
    else:
        print("\n⚠️  No standard metrics found")
else:
    print("\n⚠️  Metrics DataFrame not available")

## 4. NB2 vs ML7 Comparison

In [ ]:
# Load NB2 metrics for comparison
nb2_metrics_file = NB2_DIR / "metrics_summary.csv"
if nb2_metrics_file.exists() and df_metrics is not None:
    df_nb2 = pd.read_csv(nb2_metrics_file)
    
    # Compare macro F1
    if 'macro_f1' in df_nb2.columns and 'macro_f1' in df_metrics.columns:
        nb2_f1 = df_nb2['macro_f1'].mean()
        nb3_f1 = df_metrics['macro_f1'].mean()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        models = ['NB2 (Logistic Regression)', 'ML7 (Stage 7) (LSTM)']
        scores = [nb2_f1, nb3_f1]
        colors = ['steelblue', 'orange']
        
        bars = ax.bar(models, scores, color=colors, alpha=0.7, width=0.6)
        
        # Add value labels on bars
        for bar, score in zip(bars, scores):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{score:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=12)
        
        ax.set_ylabel('Macro F1 Score', fontweight='bold', fontsize=12)
        ax.set_title('NB2 vs ML7 (Stage 7) Performance Comparison', fontweight='bold', fontsize=14)
        ax.set_ylim([0, 1])
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        improvement = ((nb3_f1 - nb2_f1) / nb2_f1) * 100
        print(f"\n✓ Comparison complete")
        print(f"   NB2 Macro F1: {nb2_f1:.3f}")
        print(f"   ML7 (Stage 7) Macro F1: {nb3_f1:.3f}")
        print(f"   Improvement: {improvement:+.1f}%")
        
        if improvement > 0:
            print("\n   🎯 ML7 (Stage 7) (LSTM) outperforms NB2 baseline")
        else:
            print("\n   ⚠️  NB2 baseline performs better (possible overfitting in NB3)")
    else:
        print("\n⚠️  Macro F1 not found in both datasets")
else:
    print("\n⚠️  Cannot compare: NB2 or ML7 (Stage 7) metrics missing")

## 5. Sequence Predictions Analysis

In [ ]:
# Load predictions
pred_files = list(ML7_DIR.glob("*predictions*")) + list(ML7_DIR.glob("*pred*"))

if pred_files:
    pred_file = pred_files[0]
    print(f"Loading predictions from: {pred_file.name}")
    
    try:
        df_preds = pd.read_csv(pred_file)
        
        if 'date' in df_preds.columns:
            df_preds['date'] = pd.to_datetime(df_preds['date'])
        
        if 'y_true' in df_preds.columns and 'y_pred' in df_preds.columns:
            # Sample a representative period (e.g., 2024)
            if 'date' in df_preds.columns:
                df_sample = df_preds[df_preds['date'].dt.year == 2024].copy()
            else:
                df_sample = df_preds.sample(min(365, len(df_preds))).copy()
            
            if len(df_sample) > 0:
                fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True)
                
                x_vals = df_sample['date'] if 'date' in df_sample.columns else range(len(df_sample))
                
                # Ground truth
                axes[0].scatter(x_vals, df_sample['y_true'], alpha=0.6, s=15, label='True Label')
                axes[0].set_ylabel('True Label', fontweight='bold')
                axes[0].set_yticks([1, 0, -1])
                axes[0].set_yticklabels(['Stable', 'Neutral', 'Unstable'])
                axes[0].set_title('LSTM Sequence Predictions (Sample Period)', fontweight='bold', fontsize=14)
                axes[0].legend()
                axes[0].grid(True, alpha=0.3)
                
                # Predictions
                axes[1].scatter(x_vals, df_sample['y_pred'], alpha=0.6, s=15, color='orange', label='Predicted Label')
                axes[1].set_ylabel('Predicted Label', fontweight='bold')
                axes[1].set_yticks([1, 0, -1])
                axes[1].set_yticklabels(['Stable', 'Neutral', 'Unstable'])
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
                
                # Errors
                errors = (df_sample['y_true'] != df_sample['y_pred']).astype(int)
                axes[2].scatter(x_vals, errors, alpha=0.5, s=20, color='red', label='Mismatch')
                axes[2].set_ylabel('Prediction Error', fontweight='bold')
                axes[2].set_xlabel('Date' if 'date' in df_sample.columns else 'Index', fontweight='bold')
                axes[2].set_yticks([0, 1])
                axes[2].set_yticklabels(['Correct', 'Wrong'])
                axes[2].legend()
                axes[2].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
                
                accuracy = (df_preds['y_true'] == df_preds['y_pred']).mean()
                print(f"\n✓ Sequence predictions visualization complete")
                print(f"   Overall accuracy: {accuracy:.2%}")
                print(f"   Sample period: {len(df_sample)} days")
        else:
            print(f"\n⚠️  Required columns not found. Available: {df_preds.columns.tolist()}")
    except Exception as e:
        print(f"\n⚠️  Error loading predictions: {e}")
else:
    print("\n⚠️  No predictions files found")

## 6. ML7 Performance Commentary

### LSTM Architecture Overview

ML7 implements a **bidirectional LSTM** with sequence masking to handle variable-length segments:

- **Input**: 14-day windows (sliding with 7-day stride)
- **Features**: 7 physiological signals (sleep, HR, HRV, steps, screen time, etc.)
- **Architecture**: BiLSTM(64) → Dropout(0.3) → Dense(32, ReLU) → Dense(3, softmax)
- **Optimization**: Adam (lr=1e-3), class weights for imbalance
- **Training**: Early stopping (patience=10, min_delta=0.001)

### Performance Assessment

**Expected Results**:
- **Best case**: ML7 macro F1 ≈ 0.83-0.87 (3-5% improvement over NB2)
- **Typical case**: ML7 macro F1 ≈ 0.79-0.82 (marginal improvement or parity)
- **Worst case**: ML7 macro F1 < NB2 (overfitting, insufficient data)

**Key Advantages of LSTM**:
1. **Temporal dependencies**: Captures multi-day behavioral patterns
2. **Context awareness**: Considers recent history (14 days)
3. **Bidirectional**: Looks both forward and backward in time
4. **Learns representations**: Automatic feature engineering

**Challenges**:
1. **Limited data**: Single-subject N-of-1 (~2,800 days, 119 segments)
2. **Weak supervision**: Labels from PBSI heuristics (not clinical gold standard)
3. **Distribution shifts**: 8-year timeline with life events (relocation, pandemic)
4. **Overfitting risk**: Complex model with limited training examples

### Interpretation Guidelines

**If ML7 > NB2** (improvement 3-8%):
- ✅ Sequence modeling captures meaningful temporal patterns
- ✅ LSTM learns dependencies beyond static features
- ⚠️  Ensure validation loss stabilized (no overfitting)

**If ML7 ≈ NB2** (within ±2%):
- ⚠️  Temporal patterns may be weak or noisy
- ⚠️  NB2 already captures most discriminative information
- ✅ No evidence of overfitting (good generalization)

**If ML7 < NB2** (degradation >2%):
- ❌ Likely overfitting to training set
- ❌ Insufficient data for deep learning
- 💡 Consider: Reduce model complexity, increase regularization, or use NB2 as final model

### Clinical Translation

For a **real-world N-of-1 intervention**:
- **Macro F1 ≥ 0.75**: Acceptable for behavioral monitoring
- **Macro F1 ≥ 0.80**: Strong performance for hypothesis generation
- **Macro F1 < 0.70**: Insufficient for actionable insights

The deterministic pipeline ensures:
- ✅ Reproducibility (fixed seeds across TensorFlow, NumPy, Python)
- ✅ Segment-wise normalization (anti-leak safeguard)
- ✅ Calendar-based CV (temporal integrity)
- ✅ TFLite export (deployment-ready)

### Next Steps

1. **Model Selection**: Choose NB2 or ML7 based on performance
2. **TFLite Export**: Stage 8 converts best model to mobile format
3. **Report Generation**: Stage 9 creates comprehensive PDF report
4. **Clinical Validation**: Compare predictions with diary/clinical notes

## 7. Summary & Recommendations

### Pipeline Completeness Checklist

- ✅ **Stage 0-1**: Raw data extracted and aggregated
- ✅ **Stage 2-3**: Features unified and labeled (PBSI)
- ✅ **Stage 4**: Segments detected (119 segments)
- ✅ **Stage 5**: Data prepared for modeling
- ✅ **Stage 6**: NB2 baseline trained
- ✅ **Stage 7**: ML7 LSTM trained
- ⏳ **Stage 8**: TFLite export (pending)
- ⏳ **Stage 9**: PDF report generation (pending)

### Publication Checklist

For **research paper** (main.tex):
- Figure 3 (a): Use NB1 missingness bar chart
- Figure 3 (b): Use NB1 yearly summary (4-panel)
- Figure 4: Use NB1 segment timeline
- Figure 5: Use NB2 confusion matrix (normalized)
- Figure 6: Use ML7 training curves (loss + accuracy)
- Table 3: Use NB2 vs ML7 comparison (macro F1, balanced acc, kappa)

For **reproducibility**:
- ✅ All notebooks use relative paths
- ✅ Graceful handling of missing data
- ✅ Clear error messages with actionable hints
- ✅ Standard libraries only (pandas, numpy, matplotlib, seaborn)

### Final Notes

This deterministic N-of-1 pipeline demonstrates:
1. **Technical rigor**: 100% reproducible with fixed seeds
2. **Methodological soundness**: Calendar-based CV, segment-wise normalization
3. **Clinical relevance**: PBSI labels capture behavioral stability
4. **Practical utility**: TFLite export enables mobile deployment

The pipeline is ready for **thesis defense** and **journal submission**. 🎓📄